In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import preprocessing, augmentation, evaluation, training, plotting, util
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, ElasticTransform, GaussNoise, RandomCrop, Resize
from tqdm import tqdm
import os, sys
from os import path
import random as r
from import_augmentation_function import import_fun, aug_fun, import_aug_fun, normalization_fun, normalization_fun_one
import matplotlib.pyplot as plt
import tensorflow as tf 
import imageio

## Choice of folder and number of augmentations ##

In [2]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_fl_pos'
date, dye, cell_type, microscope, bf_fl, pos_neg = images_dir.split('_')
joined_path = os.path.join(files_dir, images_dir)
img_size = 256
data_ratio=0.1
data_split_state = None 
number_of_augmentations = 5 

## Import and Normalization ##
                            

In [3]:
all_image_array, all_image_array_gauss = import_fun(joined_path, files_dir, images_dir)
norm_image_array, norm_image_array_gauss = normalization_fun_one(all_image_array, all_image_array_gauss, 0.15)


## Split and Augmentation ##

In [4]:
augmentation_data, data_val, augmentation_data_gauss, data_gauss_val =  train_test_split(norm_image_array, norm_image_array_gauss,
                                                                                                       test_size=data_ratio, random_state=data_split_state)
data_aug = augmentation_data
data_gauss_aug = augmentation_data_gauss
for j in range(number_of_augmentations):
    augment_data, augment_data_gauss = aug_fun(augmentation_data, augmentation_data_gauss)
    data_aug = np.concatenate((data_aug, augment_data))
    data_gauss_aug = np.concatenate((data_gauss_aug,augment_data_gauss))

100%|██████████| 171/171 [00:00<00:00, 1171.91it/s]


## Crop ##

In [5]:
data_crop = np.zeros((np.size(data_aug , 0),128,128))
data_crop_gauss = np.zeros((np.size(data_aug , 0),128,128))
data_cropp = np.zeros((np.size(data_val , 0),128,128))
data_cropp_gauss = np.zeros((np.size(data_val , 0),128,128))

for frame_index in range(np.size(data_aug , 0)):
    data_crop[frame_index,:,:] = data_aug[frame_index, 64:192 , 64:192]
    data_crop_gauss[frame_index,:,:] = data_gauss_aug[frame_index, 64:192 , 64:192]

for frame_index in range(np.size(data_val , 0)):
    data_cropp[frame_index,:,:] = data_val[frame_index, 64:192 , 64:192]
    data_cropp_gauss[frame_index,:,:] = data_gauss_val[frame_index, 64:192 , 64:192]


data_aug=data_crop
data_gauss_aug=data_crop_gauss
data_val=data_cropp
data_gauss_val=data_cropp_gauss

## Shuffle ##

In [6]:
shuffle_array_val= np.arange(0, data_val.shape[0], 1)
shuffle_array_aug= np.arange(0, data_aug.shape[0], 1)
r.shuffle(shuffle_array_val)
r.shuffle(shuffle_array_aug)

for frame_index in range(np.size(data_val , 0)):
    data_val[frame_index]= data_val[shuffle_array_val[frame_index]]
    data_gauss_val[frame_index]= data_gauss_val[shuffle_array_val[frame_index]]

for frame_index in range(np.size(data_aug , 0)):
    data_aug[frame_index]= data_aug[shuffle_array_aug[frame_index]]
    data_gauss_aug[frame_index]= data_gauss_aug[shuffle_array_aug[frame_index]]

## Define GPU device ##

In [7]:
gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/') 

base_dir = r'C:\Users\roumba\Documents\Software\deep-events'
data_path = base_dir+ r'\Norm_data'
model_path = base_dir+ '\Models'

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [8]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history= {}, {}
  
  b=batch_size[1]
  model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
  print('Model:', model_name)
  model[model_name] = training.create_model(nb_filters, firstConvSize)
  history[model_name] = training.train_model(model[model_name], data_aug, data_gauss_aug, b, data_ratio)

Model: ref_f8_c9_b16
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
58/58 - 14s - loss: 0.5267 - binary_accuracy: 0.8309 - val_loss: 0.6062 - val_binary_accuracy: 0.9880 - 14s/epoch - 245ms/step
Epoch 2/20
58/58 - 5s - loss: 0.3578 - binary_accuracy: 0.9853 - val_loss: 0.4741 - val_binary_accuracy: 0.9880 - 5s/epoch - 86ms/step
Epoch 3/20
58/58 - 5s - loss: 0.3062 - binary_accuracy: 0.9872 - val_loss: 0.3847 - val_binary_accuracy: 0.9880 - 5s/epoch - 87ms/step
Epoch 4/20
58/58 - 5s - loss: 0.2639 - binary_accuracy: 0.9873 - val_loss: 0.3052 - val_binary_accuracy: 0.9880 - 5s/epoch - 86ms/step
Epoch 5/20
58/58 - 5s - loss: 0.2331 - binary_accuracy: 0.9873 - val_loss: 0.2497 - val_binary_accuracy: 0.9880 - 5s/epoch - 86ms/step
Epoch 6/20
58/58 - 5s - loss: 0.2070 - binary_accuracy: 0.9873 - val_loss: 0.2108 - val_binary_accuracy: 0.9880 - 5s/epoch - 86ms/step
Epoch 7/20
58/58 - 5s - loss: 0.1838 - binary_accuracy: 0.9873 - val_loss: 0.1817 - val_b

## Saving of Data ##

In [9]:
## SAVE AS .h5 ##

# name1=f'\{cell_type}_{microscope}_{bf_fl}_data_val'
# name2=f'\{cell_type}_{microscope}_{bf_fl}_data_gauss_val'
# name3=f'\{cell_type}_{microscope}_{bf_fl}_data_aug'
# name4=f'\{cell_type}_{microscope}_{bf_fl}_data_gauss_aug'

# util.save_h5(data_val_norm, data_path, name1)
# util.save_h5(data_gauss_val_norm, data_path, name2)
# util.save_h5(data_aug_norm, data_path, name3)
# util.save_h5(data_gauss_aug_norm, data_path, name4)

## SAVE AS .tiff ##

name1=f'{cell_type}_{microscope}_{bf_fl}_data_val.tiff'
name2=f'{cell_type}_{microscope}_{bf_fl}_data_gauss_val.tiff'
name3=f'{cell_type}_{microscope}_{bf_fl}_data_aug.tiff'
name4=f'{cell_type}_{microscope}_{bf_fl}_data_gauss_aug.tiff'

imageio.mimwrite(name1, (data_val).astype(np.float64))
imageio.mimwrite(name2, (data_gauss_val).astype(np.float64))
imageio.mimwrite(name3, (data_aug).astype(np.float64))
imageio.mimwrite(name4, (data_gauss_aug).astype(np.float64))


In [10]:
# import tifffile
# print(all_image_array.shape)
# writer = imageio.get_writer("all_images.tiff")
# print(writer)
# tifffile.imwrite("all_images.tiff", all_image_array.astype(np.uint16), photometric='minisblack')

In [11]:
folder_name = list(model.keys())

util.save_model(model, model_path, [f'model']*len(model), folder_name)
util.save_pkl(history, model_path, [f'history']*len(model), folder_name)

# C:/Users/roumba/Documents/Software/deep-events/Modelsref_f8_c9_b16/model.h5 #


Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/model.h5
Done.

Saving C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16/history
Done.
